<a href="https://colab.research.google.com/github/ancestor9/2026_Spring_Modeling-and-Simulation/blob/main/10_1_keras_functional_subclassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Sequential API로 만든 모델**

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(3, input_dim=4, activation='softmax'))
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 3)              │            15 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15 (60.00 B)

 Trainable params: 15 (60.00 B)

 Non-trainable params: 0 (0.00 B)

# **2. Functional API로 만든 모델**

## 1) 전결합 피드 포워드 신경망(Fully-connected FFNN)

In [2]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

inputs = Input(shape=(10,))

hidden1 = Dense(64, activation='relu')(inputs)  # <- 새로 추가
hidden2 = Dense(64, activation='relu')(hidden1) # <- 새로 추가
output = Dense(1, activation='sigmoid')(hidden2) # <- 새로 추가

model = Model(inputs, output)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,929 (19.25 KB)

 Trainable params: 4,929 (19.25 KB)

 Non-trainable params: 0 (0.00 B)

## 2) 선형 회귀(Linear Regression)

In [3]:
from tensorflow.keras import optimizers
import numpy as np # Import numpy

X = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 공부하는 시간
y = [11, 22, 33, 44, 53, 66, 77, 87, 95] # 각 공부하는 시간에 맵핑되는 성적

# Convert lists to NumPy arrays
X_np = np.array(X)
y_np = np.array(y)

inputs = Input(shape=(1,))
output = Dense(1, activation='linear')(inputs)
linear_model = Model(inputs, output)

sgd = optimizers.SGD(learning_rate=0.01)

linear_model.compile(optimizer=sgd, loss='mse', metrics=['mse'])

linear_model.fit(X_np, y_np,
                 epochs=50, # Use NumPy arrays for fitting
                 verbose=0)

## 3) 로지스틱 회귀(Logistic Regression)

In [4]:
inputs = Input(shape=(3,))
output = Dense(1, activation='sigmoid')(inputs)
logistic_model = Model(inputs, output)
logistic_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

## 4) 다중 입력을 받는 모델(model that accepts multiple inputs)

In [5]:
from tensorflow.keras.layers import concatenate

# 두 개의 입력층을 정의
inputA = Input(shape=(64,))
inputB = Input(shape=(128,))

# 첫번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
x = Dense(16, activation="relu")(inputA)
x = Dense(8, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)

# 두번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
y = Dense(64, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(8, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)

# 두개의 인공 신경망의 출력을 연결(concatenate)
result = concatenate([x.output, y.output])

z = Dense(2, activation="relu")(result)
z = Dense(1, activation="linear")(z)

model = Model(inputs=[x.input, y.input], outputs=z)

model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      8,256 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 16)        │      1,040 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 8)         │        136 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 8)         │        264 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16)        │          0 │ dense_7[0][0],    │
│ (Concatenate)       │                   │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 2)         │         34 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 1)         │          3 │ dense_11[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,813 (46.14 KB)

 Trainable params: 11,813 (46.14 KB)

 Non-trainable params: 0 (0.00 B)

## 5) RNN(Recurrence Neural Network) 은닉층 사용하기
- Input : Single feature & 50 time-step

In [6]:
from tensorflow.keras.layers import LSTM

inputs = Input(shape=(50,1))
lstm_layer = LSTM(10)(inputs)
x = Dense(10, activation='relu')(lstm_layer)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=output)
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 50, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 10)             │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 601 (2.35 KB)

 Trainable params: 601 (2.35 KB)

 Non-trainable params: 0 (0.00 B)

# **3. 서브클래싱 API로 구현한 선형 회귀**

In [7]:
import numpy as np
import tensorflow as tf

# 1. 데이터 준비
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=float).reshape(-1, 1) # Reshape X to (num_samples, 1)
y = np.array([11, 22, 33, 44, 53, 66, 77, 87, 95], dtype=float)

# 2. 서브클래싱 API로 모델 정의
class LinearRegressionModel(tf.keras.Model):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        # 선형 회귀이므로 출력 노드가 1개인 Dense 레이어 생성
        self.linear_layer = tf.keras.layers.Dense(1, input_shape=(1,))

    def call(self, inputs):
        # 모델이 실행될 때 호출되는 연산 (순전파)
        return self.linear_layer(inputs)

# 3. 모델 인스턴스 생성
model = LinearRegressionModel()

# 4. 모델 컴파일 (오차 함수와 최적화 도구 설정)
# 성적 예측(회귀) 문제이므로 Mean Squared Error(MSE)를 사용합니다.
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss='mse')

# 5. 모델 학습
print("학습 시작...")
model.fit(X, y,
          epochs=100,
          verbose=0) # 100회 반복 학습

print("학습 완료.")

# 6. 예측 결과 확인
test_hours = np.array([10], dtype=float).reshape(-1, 1) # Reshape test_hours for prediction
prediction = model.predict(test_hours)
print(f"10시간 공부했을 때 예상 성적: {prediction[0][0]:.2f}")

학습 시작...
학습 완료.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
10시간 공부했을 때 예상 성적: 107.29


# **연습문제**
## 1. 전체 features 를 입력변수로 회귀생성
## 2. 전체 features에 PCA를 적용한 후 회귀생성

In [8]:
from sklearn.datasets import make_regression
import pandas as pd

# 다중 회귀식을 위한 데이터 생성
X_multi, y_multi = make_regression(n_samples=100,
                                   n_features=10,
                                   n_informative=3,
                                   noise=0.5, random_state=42)

# 생성된 데이터를 데이터프레임으로 변환하여 확인
df_multi = pd.DataFrame(X_multi, columns=[f'feature_{i+1}' for i in range(X_multi.shape[1])])
df_multi['target'] = y_multi

df_multi

,feature_1,feature_2,feature_3,target
0,-0.815810,0.341152,-0.077102,-22.516918
1,1.441273,1.163164,-1.435862,39.422757
2,-0.342715,-0.161286,-0.802277,-9.882386
3,0.259883,-1.236951,0.781823,7.965162
4,0.865755,-0.334501,-1.200296,24.351140
...,...,...,...,...
95,0.542560,-0.465730,-0.463418,14.691784
96,-0.715304,-0.730367,0.679598,-19.888364
97,-1.062304,-0.919424,0.473592,-30.111800
98,0.250493,-0.680025,0.346448,7.311108
